# Overview
An enviroment to train and evaluate neural networks on rule learning. The language used is stated in the notions of Chomsky.
- Terminal symbols (encoded as integers)
- Nonterminalsymbols and production rules (encoded as lambda-functions)

This is how it works:
1. Section "Libraries and utilities imports libraries and defines useful functions.
2. Section "Data generation" defines functions and a class for generating the training data
3. Section "Networks" defines a function that implements different network architectures.
4. Uses the before defined functions and Derivation class.
  a. "get_training_data()" produces data using the grammar and logic for building derivations in the Derivation class the nonterminals and production rules used by "nonterms()".
  b. A loop using "evaluate()" trains all specified architectures on the data and plots the results.

In [5]:
from libs import *

In [18]:
importlib.reload(calc)

<module 'calc' from '/home/str/university/masters thesis/master-s-thesis/calc.py'>

In [124]:
inpt, outp = gnrn.dat_b(itra = [1,3], s_si = 100)
inpt_flat = []
test = copy.copy(inpt)
for i in inpt:
    inpt_flat.append(util.lflt(i))
inpt = util.pad(inpt_flat)
inpt_ten = []
for i in inpt:
    iten = torch.tensor(i)
    inpt_ten.append(iten)
inpt = []
for i in inpt_ten:
    ione = F.one_hot(i, num_classes=t_nu+8)
    inpt.append(ione)
inpt = np.array(inpt)
inpt = torch.tensor(inpt, dtype= torch.float32)
print(inpt.shape)
inpt = inpt.view(inpt.size(0),-1)
# Print results
print(f"Input: {inpt}")
print(f"Output: {outp[0]}")

Padded exmaples from X: 100%|██████████| 100/100 [00:00<00:00, 22053.23it/s]

torch.Size([100, 20, 17])
Input: tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])
Output: [1, [1, 15, 5], [7, 15, [1, 15, 5]], [3, 15, [1, 15, 5]]]


In [125]:
#dataloader
dataloader = DataLoader(dataset=inpt,shuffle=True,batch_size=100)

In [129]:
model = arch.net(inpt.shape[1],1700)
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
epochs = 1500

In [127]:
print(inpt_flat[0])

[1, 10, 11, 3, 15, 11, 1, 15, 5, 12, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
x_train_3d = inpt.view(1000, 59, 17)

In [139]:
def reve(inpt):
    inpt_reversed = []
    for i in inpt:
        reverse_x_train = torch.argmax(i, dim=1)
        reverse_x_train = reverse_x_train.tolist()
        reverse_x_train = ["[" if item == (t_nu + 2) else item for item in reverse_x_train]
        reverse_x_train = ["]" if item == (t_nu + 3) else item for item in reverse_x_train]
        reverse_x_train = [str(item) for item in reverse_x_train]
        stri = ", ".join(str(item) for item in reverse_x_train)
        stri = "[" + stri + "]"
        stri = stri.replace("[,", "[").replace(", ]", "]")
        inpt_reversed.append(stri)
    return inpt_reversed

In [152]:
def reve_y(inpt):
    inpt_reversed = []
    for i in inpt:
        _, indices = torch.max(i, dim=1)
        reverse_x_train = F.one_hot(indices, num_classes=i.shape[1]) 
        reverse_x_train = torch.argmax(i, dim=1)
        reverse_x_train = reverse_x_train.tolist()
        reverse_x_train = ["[" if item == (t_nu + 2) else item for item in reverse_x_train]
        reverse_x_train = ["]" if item == (t_nu + 3) else item for item in reverse_x_train]
        reverse_x_train = [str(item) for item in reverse_x_train]
        stri = ", ".join(str(item) for item in reverse_x_train)
        stri = "[" + stri + "]"
        stri = stri.replace("[,", "[").replace(", ]", "]")
        inpt_reversed.append(stri)
    return inpt_reversed

In [153]:
# Template from GPT-4, going to be edited:
def s_ls(y_pred, x_train):
    print(x_train.shape)
    print(y_pred.shape)
    x_3d = x_train.view(len(x_train), int(len(x_train[0])/(t_nu+8)), t_nu+8)
    print(x_3d.shape)
    y_3d = y_pred.view(len(y_pred), int(len(y_pred[0])/(t_nu+8)), t_nu+8)
    x_train_reversed = reve(x_3d)
    y_pred_reversed = reve_y(y_3d)
    print(y_pred_reversed)
#    for single_input in x_train_reversed:
 #       stri = eval(single_input)
  #      prem_index = stri.index(10)
   #     prem = stri[:prem_index]
#
 #       while compared < len():




    # Example transformation and loss computation
    # Note: adjust this to your specific requirements
    #transformed = my_custom_transformation(x_train)  # You need to define this function
    #loss = torch.mean((y_pred - transformed) ** 2)  # Example: Mean squared error
    #return loss

In [154]:
costval = []

for j in range(epochs):
    for i, x_train in enumerate(dataloader):
        #prediction
        y_pred = model(x_train)

        cost = s_ls(y_pred, x_train)
        print(f"XTRAIN:{x_train}")
        print(f"YPRED:{y_pred}")

        # Backpropagation
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if j % 50 == 0:
        print(cost)
        costval.append(cost.item())  # Use .item() to store the loss value as a number

torch.Size([100, 340])
torch.Size([100, 1700])
torch.Size([100, 20, 17])
['[9, 3, 4, 6, 2, 7, 5, 14, 3, 1, 16, 0, 5, 7, 10, 2, 16, 3, 2, 5, 5, 13, 8, 8, 1, 6], 10, 6, 14, 4, 15, [ 13, 10, 3, 6, 8, 14, 5, 16, 5, 3, 6, [ 14, 10, [ 3], 16, 6, 9, 4, 10, 10, 10, [ 15, [ 5, 8, 13, [ 14, 14, 15, 15, 3, 13, 4, 0, 8, 13, 3, 10, 9, 1, 3, 2, 10, 9, 7, 9, 6, 15, 1, 15, 4, 9, 1, 15, 7], 0, [ [ 2], 6]', '[9, 14, 8, 6, 2, 7, 5, 14, 3, 13, 16, 16, 8, 7, 10, 3, 16, 3, 2, 5, 5, 4, 8, 3, 1, 6, 3, 10, 6, 14, 10, 15, [ 13, 10, 3, 6, 8, 5, 5, 16, 5, 3, 6, [ 14, 10, [ 6], 16, 6, 9, 4, 2, 10, 10, 7, 15, [ ], 8, 13, [ 14, 14, 15, 15, 3, 2, 4, 0, 8, 13, 3, 10, 9], 3, 2, 3, 9, 7, 9, 6, 15, 1, 15, 4, 9, 1, 15, 7, 2, 0, [ 16, 2], 6]', '[9, 14, 7, 6, 2, [ 5, 14, 1, 1, 7, 0, 8, 7, 10, 3, 16, 16, 8, 9, 5, 13, 8], 1, 6, 3, 10, 6, 0, 10, 15, [ 13, 3, 3, 6, 8, 14, 5, 16, 5, 3, 10, 5, 14, 16, [ 3, [ 16, 14, 1, 4, 2, 10, 10, [ ], 16, [ 8, 13, [ 13, 14, 15, 15, 3, 13, 4, 6, 8, 13, 3, 10, 2], 3, 2, 10, 9, 7, 9, 6, 10, 1, 15

AttributeError: 'NoneType' object has no attribute 'backward'